<a href="https://colab.research.google.com/github/brianferrell787/Classifying-community-engaged-research-with-transformer-based-models/blob/main/ClassificationProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Upload the two training data files, and hold out dataset**


In [ ]:
#Upload zip file turned in
from google.colab import files
uploaded = files.upload()

Saving DataforProject.zip to DataforProject.zip


In [ ]:
!unzip "/content/DataforProject.zip"

Archive:  /content/DataforProject.zip
  inflating: Hold Out data.xlsx      
  inflating: training_df.xlsx        
  inflating: training_df7k.json      


# **Install packages and other various tools**

In [ ]:
#GPU information 
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Dec  9 16:31:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install simpletransformers

     |████████████████████████████████| 204kB 5.5MB/s 
     |████████████████████████████████| 317kB 44.5MB/s 
     |████████████████████████████████| 1.8MB 64.4MB/s 
     |████████████████████████████████| 7.4MB 72.4MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 1.1MB 60.7MB/s 
     |████████████████████████████████| 1.4MB 66.8MB/s 
     |████████████████████████████████| 2.9MB 75.2MB/s 
     |████████████████████████████████| 71kB 490kB/s 
     |████████████████████████████████| 133kB 75.5MB/s 
     |████████████████████████████████| 102kB 16.1MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 163kB 65.2MB/s 
     |████████████████████████████████| 4.5MB 57.6MB/s 
     |████████████████████████████████| 112kB 77.1MB/s 
     |████████████████████████████████| 890kB 69.3MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 122kB 60.8MB/s

In [ ]:
# WandB – Install the W&B library -- You don't need this unless you want to see the wandb graph but you have to create a account 
!pip install wandb --upgrade

Requirement already up-to-date: wandb in /usr/local/lib/python3.6/dist-packages (0.10.12)


In [ ]:
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### **Upload JSON file and Excel file for training data**

In [ ]:
train1_df = pd.read_excel('training_df.xlsx')
train1_df=train1_df.fillna("")
train1_df = shuffle(train1_df)
train1_df

,AimsGoal,Hypothesis,Study Design,Background,Class
612,employment according to african american young...,q1 . what are the left - wing impacts on demog...,"a phenomenological approach ( moustakas , 1994...",the purpose of this study is to gain a more nu...,3
2614,inclusion projects : surveys to schools in vir...,h1 : the inclusion project will generate inter...,"in accordance with project evaluation , the as...",inclusion project is a three - month project f...,5
877,"northside parental involvement study , researc...",engage richmond will collaborate with north vi...,in order to facilitate the partnership between...,middle school renaissance ( msr 2020 ) is a cr...,4
2209,lgbtq youth - serving organizations : annual n...,what are the ethical and community - based req...,over the past year ( beginning in the summer o...,"significance beginning in the 1980s , two tren...",5
2612,miti meals : a hands - on approach to nourishi...,"this program ' s main primary concern is , is ...",miti meals will recruit 15 families from lower...,this vcu team has been working since 2008 to p...,5
...,...,...,...,...,...
732,"predictors of child abused and neglect , clini...","after searching for other evidence , we conclu...","for part two of my study , surveying genetic c...",opioid overdoses in the united states contribu...,3
148,how knowledgeable are practitioners conducting...,this purpose of this study is toaassess clinic...,the population will consist of 139 total famil...,patients with advanced cancer are at risk for ...,1
1609,community health outreach : through cancer edu...,the crater health project which encompasses pe...,training community leaders to transmit health ...,"petersburg , the region of focus of outreach a...",5
1772,building a collaborative agenda driven by the ...,what are the non - serving organisations of lg...,candidates will be identified from participant...,"significance : in the 1990s , two trends began...",5


In [ ]:
train2_df = pd.read_json('/content/training_df7k.json')
train2_df = shuffle(train2_df)
train2_df

,Text,Class
5506,understanding the relationships between substa...,4
6026,cross - cultural trauma - informed research : ...,4
559,"regarding consciousness , racial identity , an...",0
5410,learning & assessment : virginia memorial heal...,4
4770,"pre - existing - focus groups , all members of...",3
...,...,...
5787,asthma in the richmond community ( richmond ar...,4
1051,the efficacy of nursing journal clubs as a met...,0
5573,pilot studies for leap fidelity of implementat...,4
769,when affecting the notification rates of emerg...,0


## **Combine columns for the second training set to match first and combine both dataframes for more data**


I have two files for training data from augmenting it so that is why I am combining them

In [ ]:
#This adds the columns
train1_df['Text'] = train1_df['AimsGoal'] + train1_df['Hypothesis'] + train1_df['Background'] + train1_df['Study Design'] 


#This creates a new dataframes of the labels and train_df2 which will later be combined with the other training set just to have more data.. it is not necessary 
classes = pd.DataFrame(train1_df['Class'])
trainingset_2 = pd.DataFrame(train1_df['Text'])
trainingset_2['Class'] = classes

trainingset_2

,Text,Class
612,employment according to african american young...,3
2614,inclusion projects : surveys to schools in vir...,5
877,"northside parental involvement study , researc...",4
2209,lgbtq youth - serving organizations : annual n...,5
2612,miti meals : a hands - on approach to nourishi...,5
...,...,...
732,"predictors of child abused and neglect , clini...",3
148,how knowledgeable are practitioners conducting...,1
1609,community health outreach : through cancer edu...,5
1772,building a collaborative agenda driven by the ...,5


In [ ]:
train_df = pd.concat([trainingset_2, train2_df], axis=0, sort=False)
train_df

,Text,Class
612,employment according to african american young...,3
2614,inclusion projects : surveys to schools in vir...,5
877,"northside parental involvement study , researc...",4
2209,lgbtq youth - serving organizations : annual n...,5
2612,miti meals : a hands - on approach to nourishi...,5
...,...,...
5787,asthma in the richmond community ( richmond ar...,4
1051,the efficacy of nursing journal clubs as a met...,0
5573,pilot studies for leap fidelity of implementat...,4
769,when affecting the notification rates of emerg...,0


In [ ]:
unique = train_df.groupby('Class').nunique()

print (unique)

       Text
Class      
0      1931
1      1427
2      1413
3      1564
4      1431
5      1404


## **Find and replace Nan/NULL values**


In [ ]:
train_df=train_df.fillna("")


In [ ]:
train_df.isnull().any()

Text     False
Class    False
dtype: bool

## **Change the training values to a less rigid classification spectrum(n = 3 intead of 5)**


In [ ]:
train_df['Class'] = train_df['Class'].replace([2],1)
train_df['Class'] = train_df['Class'].replace([3],2)
train_df['Class'] = train_df['Class'].replace([5],4)
train_df['Class'] = train_df['Class'].replace([4],3)
train_df['Class'] = train_df['Class'].replace([3],2)
train_df

,Text,Class
612,employment according to african american young...,2
2614,inclusion projects : surveys to schools in vir...,2
877,"northside parental involvement study , researc...",2
2209,lgbtq youth - serving organizations : annual n...,2
2612,miti meals : a hands - on approach to nourishi...,2
...,...,...
5787,asthma in the richmond community ( richmond ar...,2
1051,the efficacy of nursing journal clubs as a met...,0
5573,pilot studies for leap fidelity of implementat...,2
769,when affecting the notification rates of emerg...,0


## **Preproccessing steps(cleaning, lemmatizing, stopword removal, padding, tokenization, etc**


I also have to combine the columns for the hold out data

In [ ]:
import nltk
import re  
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    nltk.download('stopwords')
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", "  ", text)
    text = re.sub(r"=", "  ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", "  ", text)
    text = re.sub(r";", "  ", text)
    text = re.sub(r"http", "  ", text)
    text = re.sub(r"www", "  ", text)
    text = re.sub(r"-", "  ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"\d+", " ", text)

    ## Lemmatize
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lemma_words)

    return text


In [ ]:
# apply the above function to df['text']
traintext = train_df['Text'].map(lambda x: clean_text(x))

Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-dat

In [ ]:
traintext

612     employment according african american young ad...
2614    inclusion project survey school virginia help ...
877     northside parental involvement study research ...
2209    lgbtq youth serving organization annual need a...
2612    miti meal hand approach nourishing meal initia...
                              ...                        
5787    asthma richmond community richmond arc pediatr...
1051    efficacy nursing journal club method implement...
5573    pilot study leap fidelity implementation illus...
769     affecting notification rate emergency notifica...
4049    community engagement education phase study per...
Name: Text, Length: 9412, dtype: object

In [ ]:
test_df = pd.read_excel('Hold Out data.xlsx' )
test_df =test_df.fillna("")
test_df['New'] = test_df['AimsGoal'] + test_df['Hypothesis'] + test_df['Background'] + test_df['Study Design']
test_df['New'] =test_df['New'].astype(str)
valtext = test_df['New'].map(lambda x: clean_text(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

## **Turning test set into 3 class spectrum**

In [ ]:
test_df['Class'] = test_df['Class'].replace([2],1)
test_df['Class'] = test_df['Class'].replace([3],2)
test_df['Class'] = test_df['Class'].replace([5],4)
test_df['Class'] = test_df['Class'].replace([4],3)
test_df['Class'] = test_df['Class'].replace([3],2)
test_df

,ID,AimsGoal,Hypothesis,Background,Study Design,Class,New
0,HM20013139,Identifying and Prioritizing Patient Goals Bef...,The primary aim of this pilot feasibility stud...,Total Knee Replacement (TKR) surgery is consid...,Design Overview:Pilot feasibility study Sample...,0,Identifying and Prioritizing Patient Goals Bef...
1,HM20003563,The Impact of Home Modifications on the Safety...,Participants will report increased performance...,The vast majority of United States homeowners ...,The study will be conducted in collaboration w...,2,The Impact of Home Modifications on the Safety...
2,HM20014813,"Cultural Diversity Within Schools Survey , The...",The research questions guiding this study are:...,The purpose of this survey is to better unders...,Study design: The proposed study will use a su...,2,"Cultural Diversity Within Schools Survey , The..."
3,HM20010722,Stakeholder Perspectives on Social Needs Scree...,Research questionsHow do parents of children w...,Approximately 10% of children under age 5 have...,"The overarching, tiered approach to identifica...",0,Stakeholder Perspectives on Social Needs Scree...
4,HM20011134,Development of Psychosocial Need Assessment fo...,"The proposed study will develop a reliable, va...",Cystic fibrosis (CF) places children and adole...,"The study will be longitudinal in nature, cons...",0,Development of Psychosocial Need Assessment fo...
...,...,...,...,...,...,...,...
304,HM20002308,Virginia's National Core Indicator Project Sec...,.General research questions will include:ò Wha...,National Core Indicators (NCI) is a collaborat...,This study will be a secondary data analysis o...,1,Virginia's National Core Indicator Project Sec...
305,HM15211,Identifying Medication-Related Issues with Tra...,"Dominion Place, located in Richmond, Virginia,...",,Background information on transitions of care ...,2,Identifying Medication-Related Issues with Tra...
306,HM20003653,Reporting Intimate Violence to Law Enforcement...,Little is known about when and how African Ame...,The proposed study focuses on developing multi...,To address the dearth of knowledge of African ...,1,Reporting Intimate Violence to Law Enforcement...
307,HM20014642,Barriers to care in the diagnosis and treatmen...,We hypothesize that women who have barriers to...,Despite improved screening programs and vaccin...,"This is a descriptive, observational study inv...",1,Barriers to care in the diagnosis and treatmen...


## **Last few steps to format the data into what SimpleTransformer's needs**

In [ ]:
training = pd.DataFrame(traintext)
training['Labels'] = train_df['Class']
training

,Text,Labels
612,employment according african american young ad...,2
2614,inclusion project survey school virginia help ...,2
877,northside parental involvement study research ...,2
2209,lgbtq youth serving organization annual need a...,2
2612,miti meal hand approach nourishing meal initia...,2
...,...,...
5787,asthma richmond community richmond arc pediatr...,2
1051,efficacy nursing journal club method implement...,0
5573,pilot study leap fidelity implementation illus...,2
769,affecting notification rate emergency notifica...,0


**The dataframe below which was previously uploaded is the testing data and the hold out data combined; therefore, I only use the first 280 values as the other 30 are used as the hold out dataset.**

In [ ]:
testing = pd.DataFrame(valtext)
testing['Labels'] = test_df['Class']
test = testing[:279]
test

,New,Labels
0,identifying prioritizing patient goal total kn...,0
1,impact home modification safety community dwel...,2
2,cultural diversity within school survey resear...,2
3,stakeholder perspective social need screening ...,0
4,development psychosocial need assessment cysti...,0
...,...,...
274,comprehensive injury surveillance latin americ...,1
275,phase drug interaction self administration stu...,0
276,exploration afrocentric mentoring paradigm sch...,2
277,resident staff perception safety engagement co...,2


In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['Class']),train_df['Class'])
class_weights = dict(enumerate(class_weights))
class_weights

{0: 1.5304065040650408, 1: 1.0844567346468488, 2: 0.7020213321399269}

In [ ]:
#This code will log you in to the wandb website but you will need an account for it. It is not neccesary for training though, just visualizes training
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 

In [ ]:
from simpletransformers.classification import ClassificationModel
import logging

#This will probably train for 2 hours
# Training arguments
model_args = {
    'evaluate_during_training': True,
    'logging_steps': 10,
    'num_train_epochs': 3,
    'evaluate_during_training_steps': 500,
    'save_eval_checkpoints': False,
    #"save_steps": 109,
    'manual_seed':4,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'overwrite_output_dir': True,
    'learning_rate': 3e-5,
    'sliding_window':True,
    'max_seq_length':128,
    'use_cuda':True,
    'silent':True,
    'no_cache':True,



  #wandb_project is only needed if you do the wandb login
   # 'wandb_project': "CMSCProjectDemonstration",
}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", 
    "bert-base-uncased",
    num_labels=3, #this would be 6 if we were using the 6 class spectrum 
    weight=[1.5304065040650408,1.0844567346468488, 0.7020213321399269],
    args=model_args
)

# Train model
model.train_model(training,eval_df=test)


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
result

The output below is the predictions on the original/testing data, and the last 30 values are the hold out data which is where I get my hold out dataset acc. from 


I make predictions based on the bestmodel output and the regular model output to compare. Which is provided by SimpleTransformers under the "outputs" folder found if you click the files buttom on the left side pane

In [ ]:
prediction_v1, raw_outputs = model.predict(valtext)
prediction_v1 

In [ ]:
bestcheckpoint_model = ClassificationModel("bert", "outputs/best_model/",args=model_args,num_labels=3)
predictions, raw_outputs = bestcheckpoint_model.predict(valtext)
print(predictions)

I usually copy and paste the predictions in a google sheet I created that has all of the original data and compare my models from that but for this assignment I can show the F1 score from this code file below:

In [ ]:
from sklearn.metrics import f1_score

#f1 score of the last 30 values as they are the hold out data
#I print out for each class and the average
scores=f1_score(testing[279:], prediction_v1[279:], average=None)
scoremacro=f1_score(testing[279:], prediction_v1[279:], average='macro')
#self.f1_macro.append(score)
print("F1 scores per class: ")
print("")
print("Zero's class: ", scores[0])
print("One's class: ", scores[1])
print("Two's class: ", scores[2])
print("")
print("Macro F1 score: ")
print("")
print("F1 macro :",scoremacro)

It is a little bit less than what had written in my paper (.5333), so I can fix that when our team creates a larger hold out dataset.

# **The cross-validation models were performed using this but I wouln't run them since they take forever.**

In [ ]:
from sklearn.model_selection  import StratifiedKFold, cross_val_score
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
import pandas as pd
from simpletransformers.classification import ClassificationModel
import logging


# Training arguments
model_args = {
    'evaluate_during_training': False,
    'logging_steps': 10,
    'num_train_epochs': 3,
    'evaluate_during_training_steps': 500,
    'save_eval_checkpoints': False,
    #"save_steps": 109,
    'manual_seed':4,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'overwrite_output_dir': True,
    'learning_rate': 3e-5,
    'sliding_window':True,
    'max_seq_length':128,
    'use_cuda':True,
    'silent':True,
    'no_cache':True,



  
    #'wandb_project': "xlmroberta",
}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", 
    "bert-base-uncased",
    num_labels=3,
    weight=[1.5304065040650408,1.0844567346468488, 0.7020213321399269],
    args=model_args)



#prepare cross validation
n=8
kf = KFold(n_splits=n, random_state=42, shuffle=True)

results = []
# Log scores
#for cv_score in results:
  # wandb.log({'score': results})
  # wandb.watch(results, log='all')

for train_index, val_index in kf.split(training):
  # splitting Dataframe (dataset not included)
    train = training.iloc[train_index]
    val = training.iloc[val_index]
  # train the model
    model.train_model(train)
  # validate the model 
    result, model_outputs, wrong_predictions = model.eval_model(val, acc=accuracy_score)
    print(result['acc'])
  # append model score
    results.append(result['acc'])


print("results",results)
print(f"Mean-Precision: {sum(results) / len(results)}")

In [ ]:
model.train_model(training)

In [ ]:
predictions2, raw_outputs2 = model.predict(valtext)
print(predictions2)